In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern, hog
from scipy.stats import moment
import mahotas
from skimage import measure
from sklearn.preprocessing import StandardScaler
import ast

In [3]:
# Function to normalize an image
def normalize_image(image):
    return cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)


In [4]:
# Function to resize an image
def resize_image(image, size):
    return cv2.resize(image, size)


In [5]:
# Function to extract color moments
def extract_color_moments(image):
    moments_r = moment(image[:, :, 0].ravel(), moment=[1, 2, 3])
    moments_g = moment(image[:, :, 1].ravel(), moment=[1, 2, 3])
    moments_b = moment(image[:, :, 2].ravel(), moment=[1, 2, 3])
    return moments_r, moments_g, moments_b


In [6]:
# Function to extract Haralick texture features
def extract_haralick_features(gray_image):
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    entropy = graycoprops(glcm, 'ASM')[0, 0]
    return contrast, correlation, entropy


In [7]:
# Function to extract LBP features
def extract_lbp_features(gray_image):
    lbp = local_binary_pattern(gray_image, 8, 1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    return hist


In [8]:
# Function to extract Zernike moments
def extract_zernike_moments(gray_image):
    return mahotas.features.zernike_moments(gray_image, radius=10, degree=8)


In [9]:
# Function to extract Hu moments
def extract_hu_moments(gray_image):
    return cv2.HuMoments(cv2.moments(gray_image)).flatten()


In [10]:
# Function to apply Gabor filters and extract features
def apply_gabor_filters(gray_image):
    gabor_features = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        kernel = cv2.getGaborKernel((21, 21), 5.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        fimg = cv2.filter2D(gray_image, cv2.CV_8UC3, kernel)
        gabor_features.append(fimg.mean())
        gabor_features.append(fimg.var())
    return gabor_features


In [11]:
# Function to extract contour-based features
def extract_contour_features(binary_image, gray_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea) if contours else None
    if largest_contour is None:
        return None
    area = cv2.contourArea(largest_contour)
    perimeter = cv2.arcLength(largest_contour, True)
    x, y, w, h = cv2.boundingRect(largest_contour)
    aspect_ratio = float(w) / h
    contour_image = np.zeros_like(gray_image)
    cv2.drawContours(contour_image, [largest_contour], -1, (255), thickness=cv2.FILLED)
    eccentricity = measure.regionprops(measure.label(contour_image))[0].eccentricity
    hull = cv2.convexHull(largest_contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    solidity = area / hull_area
    return area, perimeter, aspect_ratio, eccentricity, hull_area, hull_perimeter, solidity


In [12]:
# Function to save augmented images
def save_augmented_images(image, filename, output_dir):
    augmentations = [
        ('resized', image),
        ('gray', cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)),
        ('horizontal_flip', cv2.flip(image, 1)),
        ('vertical_flip', cv2.flip(image, 0)),
        ('binary', cv2.threshold(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]),
        ('blurred', cv2.GaussianBlur(image, (5, 5), 0)),
        ('edges', cv2.Canny(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 100, 200)),
        ('dilated_edges', cv2.dilate(cv2.Canny(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 100, 200), cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)), iterations=1)),
        ('texture', cv2.Laplacian(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.CV_64F)),
        ('veins', cv2.morphologyEx(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.MORPH_BLACKHAT, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))))
    ]

    for aug_name, aug_image in augmentations:
        output_path = os.path.join(output_dir, f'{aug_name}_{filename}')
        cv2.imwrite(output_path, aug_image)
        

In [13]:
# Function to process images and extract features
def process_images(dataset_dir, output_dir, input_size):
    rows_data = []

    for subfolder in os.listdir(dataset_dir):
        subfolder_output_dir = os.path.join(output_dir, subfolder)
        os.makedirs(subfolder_output_dir, exist_ok=True)

        for root, _, files in os.walk(os.path.join(dataset_dir, subfolder)):
            for filename in files:
                if filename.endswith(('.png', '.jpg', '.JPG', '.jpeg', '.bmp')):
                    image_path = os.path.join(root, filename)
                    image = cv2.imread(image_path)
                    
                    # Normalize and resize the image
                    normalized_image = normalize_image(image)
                    resized_image = resize_image(normalized_image, input_size)
                    
                    # Save augmented images
                    save_augmented_images(resized_image, filename, subfolder_output_dir)

                    # Convert the resized image to grayscale
                    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                    
                    # Extract features
                    moments_r, moments_g, moments_b = extract_color_moments(image)
                    contrast, correlation, entropy = extract_haralick_features(gray_image)
                    lbp_features = extract_lbp_features(gray_image)
                    zernike_moments = extract_zernike_moments(gray_image)
                    hu_moments = extract_hu_moments(gray_image)
                    gabor_features = apply_gabor_filters(gray_image)

                    # Extract HSV histograms
                    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
                    hist_h = cv2.calcHist([hsv_image], [0], None, [256], [0, 256]).flatten()
                    hist_s = cv2.calcHist([hsv_image], [1], None, [256], [0, 256]).flatten()
                    hist_v = cv2.calcHist([hsv_image], [2], None, [256], [0, 256]).flatten()

                    # Extract GLCM properties
                    glcm = graycomatrix(gray_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
                    glcm_props = [graycoprops(glcm, prop).ravel()[0] for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']]

                    # Extract contour-based features
                    binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
                    contour_features = extract_contour_features(binary_image, gray_image)
                    
                    if contour_features is not None:
                        area, perimeter, aspect_ratio, eccentricity, hull_area, hull_perimeter, solidity = contour_features
                        
                        rows_data.append({
                            'Class': subfolder, 'Filename': filename,
                            'Contrast': contrast, 'Dissimilarity': glcm_props[1], 'Homogeneity': glcm_props[2], 'Energy': glcm_props[3],
                            'Correlation': glcm_props[4],'Area': area, 'Perimeter': perimeter,'Aspect_Ratio': aspect_ratio, 
                            'Eccentricity': eccentricity, 'Hull_Area': hull_area, 'Hull_Perimeter': hull_perimeter, 'Solidity': solidity,
                            'Hue_Histogram': hist_h.tolist(), 'Saturation_Histogram': hist_s.tolist(), 'Value_Histogram': hist_v.tolist(),
                            'Moments_R': moments_r.tolist(), 'Moments_G': moments_g.tolist(), 'Moments_B': moments_b.tolist(),
                            'LBP_Features': lbp_features.tolist(),'Zernike_Moments': zernike_moments.tolist(), 'Hu_Moments': hu_moments.tolist(),
                            'Gabor_Features': gabor_features
                        })
                       

    return rows_data
    

In [14]:
# Specify the dataset directory, output directory, and input size
dataset_dir = r"C:\Users\User\Desktop\project\augmented_images"
output_dir = r"C:\Users\User\Desktop\project\handcrafted_features"
input_size = (224, 224)


In [15]:
# Process the images and extract features
rows_data = process_images(dataset_dir, output_dir, input_size)

# Convert the list of dictionaries to a DataFrame
df= pd.DataFrame(rows_data)



In [16]:
# Save the DataFrame to a CSV file
df.to_csv(os.path.join(output_dir, 'output.csv'), index=False)

# Confirm the file has been saved
if os.path.exists(output_dir):
    print(f"File saved successfully")
else:
    print(f"Error: File not saved")


File saved successfully


In [17]:
print(df.dtypes)

Class                    object
Filename                 object
Contrast                float64
Dissimilarity           float64
Homogeneity             float64
Energy                  float64
Correlation             float64
Area                    float64
Perimeter               float64
Aspect_Ratio            float64
Eccentricity            float64
Hull_Area               float64
Hull_Perimeter          float64
Solidity                float64
Hue_Histogram            object
Saturation_Histogram     object
Value_Histogram          object
Moments_R                object
Moments_G                object
Moments_B                object
LBP_Features             object
Zernike_Moments          object
Hu_Moments               object
Gabor_Features           object
dtype: object


In [18]:
print(df.info)

<bound method DataFrame.info of                     Class                      Filename    Contrast  \
0      Alstonia scholaris  adjusted_image_0003_0001.JPG  109.993934   
1      Alstonia scholaris  adjusted_image_0003_0002.JPG  163.963365   
2      Alstonia scholaris  adjusted_image_0003_0003.JPG  123.790098   
3      Alstonia scholaris  adjusted_image_0003_0004.JPG  188.154168   
4      Alstonia scholaris  adjusted_image_0003_0005.JPG  135.426970   
...                   ...                           ...         ...   
15395   Terminalia arjuna    zoomed_image_0002_0193.JPG   80.025765   
15396   Terminalia arjuna    zoomed_image_0002_0217.JPG   86.074852   
15397   Terminalia arjuna    zoomed_image_0002_0218.JPG   77.640875   
15398   Terminalia arjuna    zoomed_image_0002_0219.JPG   89.022061   
15399   Terminalia arjuna    zoomed_image_0002_0220.JPG   73.734906   

       Dissimilarity  Homogeneity    Energy  Correlation     Area  \
0           6.079416     0.255005  0.091430   

In [1]:
# Scale only the single-value columns
single_value_columns = ['Contrast', 'Dissimilarity', 'Homogeneity', 'Energy', 'Correlation', 'Area', 'Perimeter',
                        'Aspect_Ratio', 'Eccentricity', 'Hull_Area', 'Hull_Perimeter', 'Solidity']
scaler = StandardScaler()
df[single_value_columns] = scaler.fit_transform(df[single_value_columns])
             

NameError: name 'StandardScaler' is not defined